In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/SMAI_Project


/content/drive/MyDrive/SMAI_Project


In [3]:
import numpy as np
import nltk
import nltk.data
import spacy
import os
import tensorflow_hub as hub
import copy

nltk.download('punkt')

sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


[nltk_data] Downloading package punkt to /home/mayur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2022-05-02 18:46:04.975106: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-05-02 18:46:04.975171: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mayur-Nitro-AN515-51): /proc/driver/nvidia/version does not exist
2022-05-02 18:46:04.993301: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 18:46:06.321767: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 34133760 exceeds 10% of free system memory.
2022-05-02 18:46:06.583603: W tensorflow/core/frame

In [4]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence]) 
  embedding = embedding.numpy()
  return embedding[0]

In [5]:
# read source file into a list of list:
def read_file(path, file_name, read_lead_only=False, read_first_doc=False):
    f = open(os.path.join(path, file_name),"r")
    lines = f.readlines()
    src_list = []
    tag="story_separator_special_tag"
    for line in lines:
        # remove tag; uncomment below for baseline
        line = line.replace(tag, "")
        # tokenzie line to sentences
        sent_list = sent_detector.tokenize(line.strip())
        src_list.append(sent_list)
    return src_list

In [6]:
def cosine_dist(A1,B1):
  A = np.array(A1)
  B = np.array(B1)
  mag_A = np.linalg.norm(A)
  mag_B = np.linalg.norm(B)

  dot_AB = np.dot(A,B)

  return (dot_AB)/(mag_A*mag_B)

cosine_dist([1,1],[1,2])

0.9486832980505138

In [7]:
def compute_cosine_mat(em_li, thres):
  length = len(em_li)
  cos_mat = np.zeros([length, length])
  conn_list = []
  for i in range(length):

    for j in range(i+1,length):

      cos_mat[i][j] = cosine_dist(em_li[index_map[i]],em_li[index_map[j]])
      cos_mat[j][i] = cos_mat[i][j]
      if(cos_mat[i][j]>thres):
        # print(i," --> ",j)
        conn_list.append((index_map[i],index_map[j]))
  return cos_mat, conn_list


In [8]:
def dfs(cur):
  if(flag[cur]==1):
    return
  # print(cur)
  flag[cur] = 1
  for key in graph_con[cur]:
    if(flag[key]==0):
      dfs(key)
  return

In [9]:
src_file = "Documents.txt"
#src_list = read_file('', src_file)[:20]

src_list = read_file('', src_file)[:100]
print("Number of instances: ", len(src_list))

Number of instances:  100


In [10]:
emb_doc = {}
conn_list_All = []
N_nodes = 0
ki=0
ct=0
index_map = {}     #  1  -->  1_9  
inv_index_map = {}  #  1_9 -->  1
for doc in src_list:
  s_li = []
  kj=0
  for sen in doc:
    emb_doc[str(ki)+"_"+str(kj)] = get_use_embedding(sen)

    index_map[ct] = str(ki)+"_"+str(kj)
    inv_index_map[str(ki)+"_"+str(kj)] = ct

    kj+=1
    ct+=1
    N_nodes+=1
  ki+=1


In [11]:

Ae, c_li = compute_cosine_mat(emb_doc,0.1)
graph_con = {}
flag = {}
for key in inv_index_map:
  graph_con[key] = []
  flag[key] = 0
for tup in c_li:
  graph_con[tup[0]].append(tup[1])
  graph_con[tup[1]].append(tup[0])

# for key in graph_con:
#   print(key," --> ",graph_con[key])

# ct=0
# for key in graph_con:
#   if(flag[key]==0 and len(graph_con[key])>0):
#     dfs(key)
#     ct+=1

# print(ct)

In [12]:
Are = copy.deepcopy(Ae)
for i in range(len(Are)):
  for j in range(len(Are[i])):
    if(i==j):
      Are[i][j] = len(graph_con[index_map[i]])
    elif(Are[i][j]>=0.1):
      Are[i][j]=-1
    else:
      Are[i][j]=0

# for i in Are:
#   print(np.sum(i))
sa=[]
for i in range(0,197):
  sa.append(len(graph_con[index_map[i]]))


In [13]:
Are = Are.astype('int64')
w, v = np.linalg.eig(Are)
v = np.real_if_close(v, tol=1)
w= np.real_if_close(w, tol=1)
eigen_asc = []

for i in range(len(w)):
  eigen_asc.append((w[i],i))
eigen_asc.sort(reverse=True)

# for i in Are:
#   print(np.sum(i))
w=np.sort(w)

for i in w:
  print(i)

#print(Are[190])

3.709524461536385e-12
51.62079934257868
59.0430949092689
64.59212073282568
69.6300587429374
80.57523571038165
81.39898313465685
84.85854953881093
87.08369626546904
88.00626591525065
88.68352307904028
90.5235683131841
90.72378465251724
91.44750483085772
92.3134264304371
92.99999999999973
93.00000000000121
93.24550069752607
93.87529274015866
94.00000000000266
95.24302611907436
96.00000000000146
96.24029383518653
97.43113859905274
97.9696250106575
98.61058063720904
99.09287411808121
100.06565000330396
101.32549124318524
101.80840297490482
102.65911309661767
103.7868209799678
104.81267327697604
106.83977953245608
107.57840107437976
108.00000000000038
109.79839245227123
111.26908271964366
112.93237681679767
113.37619686216463
114.66559980532816
115.98618805776155
119.15549195169889
120.51969593567861
120.78293791355105
122.38688230479836
123.03178673277726
123.54285046264704
124.17245845412168
124.77867277716659
125.00000000000269
125.24706691016925
127.24339978904104
127.30188264194832
127

In [14]:
print((Are==Are.T).all())

True


In [15]:
k = 10
li_k_vec = []
ind_eigv = [] 

for i in range(k):
  ind_eigv.append(eigen_asc[i][1])
  li_k_vec.append(v[eigen_asc[i][1]])

In [16]:
N_nodes

2048

In [17]:
N_coor = np.array(li_k_vec).T

In [18]:
print(N_coor.shape)

(2048, 10)


In [19]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=k, n_init=10, random_state=0)
kmeans.fit(N_coor)

KMeans(n_clusters=10, random_state=0)

In [20]:
zr=np.zeros(10)
for i in kmeans.labels_:
  zr[i]=zr[i]+1
for i in zr:
  print(i)

2009.0
1.0
3.0
1.0
1.0
1.0
3.0
16.0
12.0
1.0


In [31]:
import spacy 
spacy_=spacy.load("en_core_web_sm")

def pos_tag(sentences):
  out=[]
  for sentence in sentences:
    sentence =sentence.replace("/","")
    word_dict=spacy_(sentence)
    temp=[]
    for word in word_dict:
      tagg=word.text+"/"+word.tag_
      temp.append(tagg)
    out.append(' '.join(temp))
  return out


In [32]:
st=["hello how are you?","long time i have been waiting for you"]
print(pos_tag(st))

['hello/UH how/WRB are/VBP you/PRP ?/.', 'long/JJ time/NN i/PRP have/VBP been/VBN waiting/VBG for/IN you/PRP']


In [2]:
import takahe

In [ ]:
def compression(sentences):
  compresser = takahe.word_graph( sentences,nb_words = 6,lang = 'en',punct_tag =".")
  candidates = compresser.get_compression(10)
  for cummulative_score, path in candidates:
	  normalized_score = cummulative_score / len(path)
	  print round(normalized_score, 3), ' '.join([u[0] for u in path])